# FDK Reconstruction

In [2]:
import os, sys, glob
import numpy as np
import logging
import time
import matplotlib.pyplot as plt

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    my_directory = '[[MY_Google_drive_directory]]'
    os.chdir(f'/content/drive/MyDrive/{my_directory}')
    !pip install pycuda

from Reconstruction_pycuda import Reconstruction

pi = np.pi
logging.basicConfig(level = logging.INFO)
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

In [3]:
nn = 256
NumberOfDetectorPixels = [512, 384]
NumberOfViews = 90
FilterType = 'ram-lak'

savedir = 'results'
if not os.path.exists(savedir):
    os.makedirs(savedir)

params = {'SourceInit': [0, 1000.0, 0], 'DetectorInit': [0, -500.0, 0], 'StartAngle': 0, 'EndAngle': 2 * pi,
          'NumberOfDetectorPixels': NumberOfDetectorPixels, 'DetectorPixelSize': [0.5, 0.5], 'NumberOfViews': NumberOfViews,
          'ImagePixelSpacing': [0.5, 0.5, 0.5], 'NumberOfImage': [nn, nn, nn], 'PhantomCenter': [0, 0, 0],
          'RotationOrigin': [0, 0, 0], 'ReconCenter': [0, 0, 0], 'Method': 'Distance', 'FilterType': FilterType,
          'cutoff': 1, 'GPU': 1, 'DetectorShape': 'Flat', 'Pitch': 0, 'DetectorOffset': [0, 0]}

R = Reconstruction(params)

filename = os.path.join('phantoms', f'Shepp_Logan_3d_({nn}x{nn}x{nn}).raw')
R.LoadRecon(filename)

ph = R.image

### Analytic projection

In [9]:
start_time = time.time()
R.forward()
log.info(f'Forward {time.time() - start_time:.3f} sec')
proj0 = np.copy(R.proj)
R.SaveProj(os.path.join(savedir, f'proj_SheppLogan_({R.proj.shape[2]}x{R.proj.shape[1]}x{R.proj.shape[0]}).raw'))

INFO:__main__:Forward 2.695 sec


In [ ]:
plt.figure()
plt.imshow(proj0[:, 250, :], cmap='gray')
plt.show()

### Backprojection

In [10]:
start_time = time.time()
R.backward()
log.info(f'Backward: {time.time() - start_time:.3f} sec')
R.SaveRecon(os.path.join(savedir, f'BP_SheppLogan_({R.image.shape[2]}x{R.image.shape[1]}x{R.image.shape[0]}).raw'))
bp = np.copy(R.image)

INFO:__main__:Backward: 1.876 sec


In [ ]:
plt.figure()
plt.imshow(bp[81, :, :], cmap='gray')
plt.show()

### FDK Reconstruction

In [ ]:
R.image = np.zeros(params['NumberOfImage'], dtype=np.float32)
start_time = time.time()
R.Filtering()
R.backward()
log.info(f'FDK: {time.time() - start_time:.3f} sec')
R.SaveRecon(os.path.join(savedir, f'Recon_SheppLogan_fdk_({R.image.shape[2]}x{R.image.shape[1]}x{R.image.shape[0]}).raw'))

In [ ]:
plt.figure()
plt.imshow(R.image[81, :, :], cmap='gray')
plt.show()